# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.9 Function calling高级函数封装与开发流程优化

&emsp;&emsp;在上一小节中，我们已经完整介绍了Chat模型的Function calling功能实现流程，本节我们将进一步围绕Chat模型的Function calling功能进行更高层次的函数封装，以降低实现该功能的门槛，提高开发效率。同时，我们也将进一步介绍外部工具API获取方法，以OpenWeather为例，尝试通过Function calling功能调用OpenWeather API实现实时天气查询。而有了这些内容做铺垫，下一小节我们将进一步介绍实用性更强的基于Chat模型Function calling功能的AI应用程序开发案例。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202307131805614.png" alt="Function calling流程总结 (2)" style="zoom:33%;" />

然后需要补充定义下一小节定义的关键变量：

In [45]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests

In [2]:
# 创建一个DataFrame
df = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]})

df_str = df.to_string()

data = io.StringIO(df_str)

df_new = pd.read_csv(data, sep='\s+', index_col=0)

In [3]:
def chen_ming_algorithm(data):
    """
    陈明算法函数，该函数定义了一种特殊的数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：陈明函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    df_new = pd.read_json(data)
    res = np.sum(df_new, axis=1) - 1
    return res.to_json(orient='records')

In [4]:
available_functions = {
            "chen_ming_algorithm": chen_ming_algorithm,
        }

In [5]:
chen_ming_function = {"name": "chen_ming_algorithm",
                      "description": "用于执行陈明算法的函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行陈明算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }

In [6]:
functions = [chen_ming_function]
functions

[{'name': 'chen_ming_algorithm',
  'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
   'required': ['data']}}]

- Function calling功能实现流程优化基本思路

&emsp;&emsp;对于原始的Function calling实现流程而言，尽管过程清晰，但要完整跑通一个流程所涉及到的代码环节较多，在高频调用该功能的场景中，这个复杂的代码流程会极大程度影响使用和开发效率。因此，接下来我们考虑如何对上述流程进行优化。总的来说，优化的方向有两个：
- 第一个优化方向就是优化functions参数的编写效率。根据上述手动实现Function calling功能的过程不难发现，将外部函数信息输入到functions参数中的过程非常复杂，需要涉及到大量的JSON Schema编写过程，并且，如果函数库中包含大量函数，逐个编写JSON Schema会非常复杂且对于初学者而言非常容易出错，因此我们希望能找到一种方法，大幅降低Chat模型读取外部函数的门槛。例如，能否当我们编写完陈明函数后模型就能让大语言模型自动识别这个函数并创建相应的functions参数，而不用手动对其进行编写；


- 第二个优化方向则是围绕second response流程进行优化，一个非常朴素的交互需求是，我们并不关心向大模型提问时中间有几次调用模型的过程，我们只希望在一次对话中快速完成需求，即无论是否进行外部函数调用，我们都希望能够在一次代码交互过程中完成对话任务。因此可以考虑对对话流程进行更高层次的封装，若对话过程需要调用Function calling，也能自动执行second response，并最终在一次对话中返回最终结果。

接下来我们就尝试编写一些辅助开发的函数，来完成上述两个方面的优化需求。

### 1.functions参数自动编写函数

&emsp;&emsp;这里我们首先需要简单验证大语言模型是否具备自主识别函数功能并进行functions参数编写的能力。首先，通过观察不难发现，functions参数其实是非常一类高度结构化的参数，而参数中的核心信息其实都来源于函数本身:

In [14]:
functions

[{'name': 'chen_ming_algorithm',
  'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
   'required': ['data']}}]

In [15]:
chen_ming_function

{'name': 'chen_ming_algorithm',
 'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
 'parameters': {'type': 'object',
  'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
  'required': ['data']}}

In [13]:
chen_ming_algorithm?

Signature: chen_ming_algorithm(data)
Docstring:
陈明算法函数，该函数定义了一种特殊的数据集计算过程
:param data: 必要参数，要求字符串类型，表示带入计算的数据表
:return：陈明函数计算后的结果，返回结果为json字符串类型对象
File:      c:\users\administrator\appdata\local\temp\ipykernel_8980\2241329342.py
Type:      function

因此，functions参数的编写其实本质上上就是一个翻译的过程，将函数原始的说明信息翻译成functions参数要求的格式。而模型是否具备“翻译能力”呢？这其实取决于大语言模型对JSON Schema格式的熟悉程度，我们可以通过询问的方式验证大模型是否具备这方面知识：

In [18]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "user", "content": "请问什么是JSON Schema？"}
  ]
)

response.choices[0].message['content']

'JSON Schema是一种用于描述JSON数据格式的元数据标准。本质上，它提供了一种描述数据格式的语言。这可以用于验证一个JSON对象的结构是否符合某个预设的模式，也可以用来对JSON对象进行编辑，以保证输入的数据格式正确。'

能够看出，Chat模型是具备JSON Schema相关知识储备的。由此不难判断，只要我们详细的编写每个函数的函数说明，并且通过合理的提示让模型理解functions参数结构，同时借助模型本身对JSON Schema的理解，是能够让Chat模型自主读取并创建函数的functions参数的。具体实现过程如下。

- 函数说明提取

&emsp;&emsp;首先我们首先可以通过inspect库中的getdoc方法来将函数说明提取为字符串：

In [44]:
import inspect

In [16]:
inspect.getdoc(chen_ming_algorithm)

'陈明算法函数，该函数定义了一种特殊的数据集计算过程\n:param data: 必要参数，要求字符串类型，表示带入计算的数据表\n:return：陈明函数计算后的结果，返回结果为json字符串类型对象'

In [17]:
print(inspect.getdoc(chen_ming_algorithm))

陈明算法函数，该函数定义了一种特殊的数据集计算过程
:param data: 必要参数，要求字符串类型，表示带入计算的数据表
:return：陈明函数计算后的结果，返回结果为json字符串类型对象


In [18]:
function_description = inspect.getdoc(chen_ming_algorithm)
function_description

'陈明算法函数，该函数定义了一种特殊的数据集计算过程\n:param data: 必要参数，要求字符串类型，表示带入计算的数据表\n:return：陈明函数计算后的结果，返回结果为json字符串类型对象'

> inspect库中包含了非常多用于辅助验证函数功能的函数，能够非常高效的执行获取函数参数、函数源码、函数说明等功能。

- 测试能够将函数参数格式以JSON Schema类型呈现

&emsp;&emsp;然后测试能否让Chat模型通过读取这段函数说明，正确识别函数参数及参数结构，并用JSON Schema格式进行说明：

In [22]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "以下是陈明函数的函数说明：%s" % function_description},
    {"role": "user", "content": "请帮我编写一个JSON Schema对象，用于说明陈明函数的参数输入规范。输出结果要求是JSON Schema格式的JONS类型对象，不需要任何前后修饰语句。"}
  ]
)
response.choices[0].message['content']

'{\n  "$schema": "http://json-schema.org/draft-07/schema#",\n  "title": "ChenMing Function Parameter",\n  "type": "object",\n  "properties": {\n    "data": {\n      "description": "要求字符串类型，表示带入计算的数据表",\n      "type": "string"\n    }\n  },\n  "required": ["data"]\n}'

Chat函数的输出结果是JSON格式对象，我们可以通过json.loads方法将其转化为python对象：

In [23]:
json.loads(response.choices[0].message['content'])

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'ChenMing Function Parameter',
 'type': 'object',
 'properties': {'data': {'description': '要求字符串类型，表示带入计算的数据表',
   'type': 'string'}},
 'required': ['data']}

In [33]:
chen_ming_function

{'name': 'chen_ming_algorithm',
 'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
 'parameters': {'type': 'object',
  'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
  'required': ['data']}}

In [38]:
chen_ming_function['parameters']

{'type': 'object',
 'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
 'required': ['data']}

对比手动编写的结果不难发现，模型能够根据函数的参数说明正确识别陈明函数的参数格式，并输出对应的JSON Schema对象。

> 相比手动编写的JSON Schema对象，Chat模型输出的JSON Schema对象多出来的'$schema'关键词表示的是元关键词，用于申明当前JSON Schema版本，而'additionalProperties'关键词则表示不存在另一种输入格式，这两个关键词对所描述的对象结构类型并没有任何影响。

- 编写自动编写functions参数函数

&emsp;&emsp;接下来，我们试着通过合理的提示，让模型能够自动编写functions参数，还是一样的，我们可以通过chen_ming_algorithm.__name__自动提取函数名称：

In [83]:
function_name = chen_ming_algorithm.__name__
function_name

'chen_ming_algorithm'

以下是Chat模型提示过程：

In [123]:
system_prompt = '以下是某的函数说明：%s' % function_description
user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
               1.字典总共有三个键值对；\
               2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
               3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
               4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
               5.输出结果必须是一个JSON格式的字典，且不需要任何前后修饰语句' % function_name

In [75]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]
)
response.choices[0].message['content']

'{\n    "name": "chen_ming_algorithm",\n    "description": "陈明算法函数，该函数定义了一种特殊的数据集计算过程",\n    "parameters": {\n        "$schema": "http://json-schema.org/draft-07/schema#",\n        "title": "陈明算法函数参数",\n        "type": "object",\n        "properties": {\n            "data": {\n                "description": "字符串类型的数据表",\n                "type": "string"\n            }\n        },\n        "required": ["data"]\n    }\n}'

In [76]:
json.loads(response.choices[0].message['content'])

{'name': 'chen_ming_algorithm',
 'description': '陈明算法函数，该函数定义了一种特殊的数据集计算过程',
 'parameters': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'title': '陈明算法函数参数',
  'type': 'object',
  'properties': {'data': {'description': '字符串类型的数据表', 'type': 'string'}},
  'required': ['data']}}

> 当然，上述过程（按照格式输出文本语义理解的内容）对于大语言模型来说其实是非常简单的推理过程，这个提示过程也可以借助Few-shot来执行。并且在JSON Schema对象编写时，Few-shot效果会明显好于当前的Zero-shot过程。

而这个对象，其实也就是我们此前编写的chen_ming_function对象。

In [58]:
chen_ming_function

{'name': 'chen_ming_algorithm',
 'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
 'parameters': {'type': 'object',
  'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
  'required': ['data']}}

&emsp;&emsp;接下来，我们对上述流程进行更高层次的封装，即编写一个自动输出functions参数的函数：

In [39]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = openai.ChatCompletion.create(
                              model="gpt-4-0613",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            functions.append(json.loads(response.choices[0].message['content']))
        return functions
    
    max_attempts = 3
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [38]:
chen_ming_algorithm.__name__

'chen_ming_algorithm'

这里需要注意，我们实际上是把函数功能的主题封装在functions_generate这个内嵌函数中，然后外层函数主要控制报错时的处理流程：即如果函数执行时报错，则三次内报错都会反复调用执行functions_generate，三次报错之后则会直接停止运行。这里之所以要设置多次报错仍然反复执行，是因为哪怕user_prompt中明确指出“只输出这个字典即可，前后不需要任何前后修饰或说明的语句”，但模型仍然可能会输出前后说明文字，此时我们是无法直接使用functions.append(json.loads(response.choices[0].message['content']))来提取functions字典的，但这只是小概率事件，再次进行相同问题的提问，输出的结果大概率不会再包含前后修饰语句，functions_generate即可正常运行。因此这里设置了一旦报错最多反复三次进行运行的流程：

> 当然，这个问题也可以通过Few-shot来解决。

In [25]:
functions_list = [chen_ming_algorithm]

In [26]:
functions = auto_functions(functions_list)

In [27]:
functions

[{'name': 'chen_ming_algorithm',
  'description': '陈明算法函数，该函数定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string',
     'description': '必要参数，要求字符串类型，表示带入计算的数据表'}},
   'required': ['data']}}]

接下来测试将其带入Chat模型，验证模型是否能顺利执行Function calling功能：

In [28]:
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [29]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行陈明算法"}
]

response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x163ae338b80> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "chen_ming_algorithm",
    "arguments": "{\n\"data\": \"x1  x2\\n0   1   3\\n1   2   4\"\n}"
  }
}

能够发现模型返回结果中存在function_call，则说明模型完成了外部模型的挑选，说明模型顺利执行了Function calling功能。也进而说明自动functions参数编写函数切实有效。

&emsp;&emsp;接下来我们进一步测试，当我们添加多个外部函数时，auto_functions函数能否顺利的依次翻译这些函数说明，并组成functions列表，同时在多个函数情况下，模型能否根据实际对话需求智能选择外部函数：

In [70]:
def zhao_min_algorithm(data):
    """
    赵敏算法函数，该函数定义了一种特殊的数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：赵敏函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    df_new = pd.read_json(data)
    res = np.sum(df_new, axis=1) + 1
    return res.to_json(orient='records')

In [34]:
functions_list = [chen_ming_algorithm, zhao_min_algorithm]

In [35]:
functions = auto_functions(functions_list)
functions

[{'name': 'chen_ming_algorithm',
  'description': '陈明算法函数，该函数定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string',
     'description': '要求字符串类型，表示带入计算的数据表'}},
   'required': ['data']}},
 {'name': 'zhao_min_algorithm',
  'description': '赵敏算法函数，该函数定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string',
     'description': '必要参数，要求字符串类型，表示带入计算的数据表'}},
   'required': ['data']}}]

In [100]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行赵敏算法"}
]

response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )
response["choices"][0]["message"]

<OpenAIObject at 0x227ec87ce50> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n\"data\": \"x1  x2\\n0   1   3\\n1   2   4\"\n}",
    "name": "zhao_min_algorithm"
  },
  "role": "assistant"
}

能够发现，在添加了赵敏算法之后，auto-functions函数能够顺利输出正确结果，并且模型也能够根据不同的提示，智能筛选外部函数。例如此时在同时输入了陈明算法和赵敏算法作为外部函数的时，当我们向模型提问“执行赵敏算法”时，模型能够正确识别调用zhao_min_algorithm。

### 2.外部函数调用时两轮response自动应答函数

&emsp;&emsp;接下来，我们继续考虑围绕对话流程进行优化，即将Function calling执行时的多轮对话封装在一个函数中。这里需要涉及到外部函数库字典创建，该字典要求一个键值对代表一个函数，每个键值对的Key表示函数名字符串，对应的Value表示对应的函数。对于此前的functions_list里面包含的两个函数，我们可以使用如下方式创建这个函数库字典：

In [103]:
functions_list

[<function __main__.chen_ming_algorithm(data)>,
 <function __main__.zhao_min_algorithm(data)>]

In [105]:
function_dict = {func.__name__: func for func in functions_list}
function_dict

{'chen_ming_algorithm': <function __main__.chen_ming_algorithm(data)>,
 'zhao_min_algorithm': <function __main__.zhao_min_algorithm(data)>}

具体函数的定义流程如下：

In [40]:
def run_conversation(messages, functions_list=None, model="gpt-4-0613"):
    """
    能够自动执行外部函数调用的Chat对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response["choices"][0]["message"]
        final_response = response_message["content"]
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        functions = auto_functions(functions_list)
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # first response
        response = openai.ChatCompletion.create(
                        model=model,
                        messages=messages,
                        functions=functions,
                        function_call="auto")
        response_message = response["choices"][0]["message"]

        # 判断返回结果是否存在function_call，即判断是否需要调用外部函数来回答问题
        if response_message.get("function_call"):
            # 需要调用外部函数
            # 获取函数名
            function_name = response_message["function_call"]["name"]
            # 获取函数对象
            fuction_to_call = available_functions[function_name]
            # 获取函数参数
            function_args = json.loads(response_message["function_call"]["arguments"])
            # 将函数参数输入到函数中，获取函数计算结果
            function_response = fuction_to_call(**function_args)

            # messages中拼接first response消息
            messages.append(response_message)  
            # messages中拼接函数输出结果
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )  
            # 第二次调用模型
            second_response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
            )  
            # 获取最终结果
            final_response = second_response["choices"][0]["message"]["content"]
        else:
            final_response = response_message["content"]
    
    return final_response

接下来测试函数效果：

In [40]:
messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请在data上执行赵敏算法'}]

In [42]:
functions_list

[<function __main__.chen_ming_algorithm(data)>,
 <function __main__.zhao_min_algorithm(data)>]

In [43]:
run_conversation(messages = messages, functions_list = functions_list)

'执行赵敏算法后得到的结果是：\n```\n0    5\n1    7\n```'

In [46]:
messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请在data上执行陈明算法'}]
run_conversation(messages = messages, functions_list = functions_list)

'执行了陈明算法后，数据集结果如下：  \n\n   x1  x2\n0   3  \n1   5'

In [48]:
messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请帮我介绍下data数据集'}]
run_conversation(messages = messages, functions_list = functions_list)

'这是一个简单的二维数据集，它由两列数据x1和x2组成。数据集中有两行数据，对应的值分别是（1，3）和（2，4）。这样的数据集通常可以用于简单的二维数值分析或可视化。'

In [55]:
messages = [
        {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
        {"role": "user", "content": '请在data上执行陈明算法'}]
run_conversation(messages = messages)

'抱歉，关于"陈明算法"的信息我无法找到，它可能属于一个非常特殊或私有的算法。如果你能提供更多关于它的具体信息，例如它的主要功能，适用场景或是一些具体的操作步骤等，我可能可以帮助到你。\n\n或者，你可以最后是指其他流行的ML算法（例如，线性回归，逻辑回归，决策树，k-means，SVM，神经网络等）吗？在这个数据集上，我可以调用这些算法来进行数据分析和预测。'

能够发现，模型能够非常顺利的调用外部函数并围绕当前问题进行准确回答。至此，我们就完成了既定的Function calling功能执行过程的代码优化，通过借助run_conversation函数，我们只需设置messages和外部函数列表，即可让模型在回答时有选择性的选择外部函数进行回答，全程无需手动进行调整。

### 3.基于run_conversation的多轮对话函数

&emsp;&emsp;当然，我们还可以更进一步，在run_conversation基础之上，再封装一个可以执行多轮对话的函数。我们可以直接在Ch.7结尾定义的多轮对话函数基础上进行修改，修改后的函数如下：

In [58]:
system_message=[{"role": "system", "content": "你是以为乐于助人的助手。"}]

In [59]:
system_message.append({"role": "user", "content": "你好呀"})

In [60]:
system_message

[{'role': 'system', 'content': '你是以为乐于助人的助手。'},
 {'role': 'user', 'content': '你好呀'}]

In [41]:
def chat_with_model(functions_list=None, 
                    prompt="你好呀", 
                    model="gpt-4-0613", 
                    system_message=[{"role": "system", "content": "你是以为乐于助人的助手。"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                                    functions_list=functions_list, 
                                    model=model)
        
        
        print(f"模型回答: {answer}")

        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

接下来测试问答效果，首先是不带入functions_list的问答过程：

In [64]:
chat_with_model()

模型回答: 你好，很高兴和你交谈。有什么我可以帮助你的吗？


您还有其他问题吗？(输入退出以结束对话):  请帮我介绍下陈明算法


模型回答: 很抱歉，我没有找到关于"陈明算法"的准确信息。可以没有描述错误或者这个算法不那么常见或者被广泛的知道。如果你有更多陈明算法的详细信息，我可以帮你更深入的研究。我随时准备助你一臂之力。


您还有其他问题吗？(输入退出以结束对话):  退出


能够发现，模型并不知道外部函数库的任何信息。接下来，我们进一步测试带入functions_list之后模型问答效果：

In [69]:
chat_with_model(functions_list, prompt="你好")

模型回答: 你好！有什么我可以帮你的吗？


您还有其他问题吗？(输入退出以结束对话):  请问什么是陈明函数？


模型回答: 陈明算法是一种特殊的数据集计算过程，这个函数需要一串字符串作为数据表。直接的功能和详细的算法过程可能因具体的应用背景和需求而有所不同。大体上，它的主要作用是通过对输入的字符串数据表进行复杂的操作和运算，从而得到一个结果输出。可能的操作包括但不限于排序、分类、筛选等。具体的计算方法和输出结果需要根据陈明算法的具体实现来确定。


您还有其他问题吗？(输入退出以结束对话):  那赵敏函数呢？


模型回答: 赵敏算法是一种特殊的数据集计算过程，此函数需要一个字符串类型的输入，这个输入代表的是带入计算的数据表。具体的计算过程和结果取决于赵敏算法的内部实现。请注意，使用此函数需要精确的数据输入，以获得可靠的计算结果。


您还有其他问题吗？(输入退出以结束对话):  好的，我还想了解下无忌函数


模型回答: 抱歉，目前我们只定义了陈明算法和赵敏算法两种计算函数。"无忌函数"可能并不存在或者在这里并未定义，所以我无法为您提供相关详细信息。如果有更多其他问题，请随时向我询问。


您还有其他问题吗？(输入退出以结束对话):  退出


能够发现，此时模型不仅能够顺利识别外部函数库的陈明函数和赵敏函数，同时甚至能够对外部函数库整体情况进行总结。由此也能确定，Chat模型对外部函数的识别是建立在外部函数库全量信息的理解基础上的，而这些理解不仅可以用于具体某个外部函数的问答，也可以用于对整个函数库进行描述。

- gtpLearning.py函数库封装

&emsp;&emsp;最后，考虑到我们后续需要反复用到本节定义的这些函数，为了更方便的调用，我们可以在本地创建一个名为gptLearning.py的py文件用于保存这些代码，并在必要时随时通过import方法对其导入。gptLearning.py目前包含auto_functions、run_conversation、chat_with_model三个函数以及必要的库，同学们可以自行在本地创建同名py文件，也可以直接下载课件里的gptLearning.py文件并放在jupyter的当前工作目录下，方便后续进行导入。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/86d693dc06da838abcbb1bbc6a47ca3.png" alt="86d693dc06da838abcbb1bbc6a47ca3" style="zoom:33%;" />

py文件准备好了之后，之后即可按照如下方式导入这些函数：

In [99]:
import gptLearning as gl

In [100]:
gl?

Type:        module
String form: <module 'gptLearning' from 'E:\\work\\大模型课程课件\\gptLearning.py'>
File:        e:\work\大模型课程课件\gptlearning.py
Docstring:   <no docstring>

In [102]:
gl.auto_functions?

Signature: gl.auto_functions(functions_list)
Docstring:
Chat模型的functions参数编写函数
:param functions_list: 包含一个或者多个函数对象的列表；
:return：满足Chat模型functions参数要求的functions对象
File:      e:\work\大模型课程课件\gptlearning.py
Type:      function

In [105]:
gl.run_conversation?

Signature: gl.run_conversation(messages, functions_list=None, model='gpt-4-0613')
Docstring:
能够自动执行外部函数调用的Chat对话模型
:param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
:param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
:param model: Chat模型，可选参数，默认模型为gpt-4
:return：Chat模型输出结果
File:      e:\work\大模型课程课件\gptlearning.py
Type:      function

In [104]:
gl.chat_with_model?

Signature:
gl.chat_with_model(
    functions_list=None,
    prompt='你好呀',
    model='gpt-4-0613',
    system_message=[{'role': 'system', 'content': '你是以为乐于助人的助手。'}],
)
Docstring: <no docstring>
File:      e:\work\大模型课程课件\gptlearning.py
Type:      function

也可以按照如下方式一次性导入全部所需函数：

In [98]:
from gptLearning import *

&emsp;&emsp;至此，我们围绕Function calling功能的实现流程的优化就全部完成，接下来我们将目光转向外部函数库的构建过程中，尝试获取一些开源工具的API并将其封装为外部函数，借助Function calling功能进行调用，以实现Chat模型功能的大幅拓展。